In [ ]:
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [ ]:
data = open('20200913').read()
contents = []
for i, line in enumerate(data.split("\n")):
    if line != "":
        content = line.split("\t")
        contents.append([content[i] for i in (3,4,6)])

title = ["搜索提问","新闻标题","时间"]
Data = pd.DataFrame(data=contents,columns=title)

In [ ]:
Data.head()

,搜索提问,新闻标题,时间
0,谭明明父母,玛莎拉蒂撞人事件：谭明明称赔457万。被害家属：判死刑一分不要！,2020-09-14 16:15:09
1,不要赔偿判死刑,玛莎拉蒂撞人事件：谭明明称赔457万。被害家属：判死刑一分不要！,2020-09-14 06:27:15
2,谭明明,玛莎拉蒂撞人事件：谭明明称赔457万。被害家属：判死刑一分不要！,2020-09-14 08:47:47
3,谭明明,玛莎拉蒂撞人事件：谭明明称赔457万。被害家属：判死刑一分不要！,2020-09-14 22:09:26
4,谭明明,玛莎拉蒂撞人事件：谭明明称赔457万。被害家属：判死刑一分不要！,2020-09-14 04:48:10


In [ ]:
def json_generator(dataframe):
    json_doc = []
    for data_id,(title,time) in enumerate(zip(dataframe["新闻标题"],dataframe["时间"])):
        json_doc.append({"_index":"新闻","_type":"标题时间","_id":data_id,"_source":{"新闻标题":title,"时间":time}})
    return json_doc    

In [ ]:
def insert_batch_data(json_doc):
    for i in range(0,len(json_doc),1000000):
        if len(json_doc)-i < 1000000:
            helpers.bulk(es,json_doc[i:])
        else:
            helpers.bulk(es,json_doc[i:i+1000000])

In [ ]:
es = Elasticsearch(HOST="http://localhost",PORT=9200)
es = Elasticsearch()

In [ ]:
es.count(index="新闻",doc_type="标题时间")

{'count': 6503384,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [ ]:
def search_query(index,doc,column,content,start,num):
    """ 
    index - 索引 doc - 文档类型 column - 列名称 content - 关键词
    start - 从“第几条”开始查询 num - 查询条数
    """
    res = es.search(index=index,doc_type=doc,body={"from":start,"size":num,"query":{"match":{column:content}}})
    if res["hits"]["hits"] == []:
        print("没有匹配到的信息，请重新输入")
    info_lists = [info["_source"] for info in res["hits"]["hits"]]
    info_dataframe = pd.DataFrame(info_lists)
    return info_dataframe

In [ ]:
search_query("新闻","标题时间","新闻标题","G5高速",0,10)

,新闻标题,时间
0,G5成绵扩容高速、绵苍高速、中遂高速三个项目宣布开工,2020-09-14 07:45:52
1,G5京昆高速公路交通管控和限载、限速……,2020-09-14 20:55:56
2,G5京昆高速公路交通管控和限载、限速……,2020-09-14 23:17:53
3,G5京昆高速公路交通管控和限载、限速……,2020-09-14 17:34:05
4,56天！G5京昆高速公路实行交通管制,2020-09-14 23:17:04
5,56天！G5京昆高速公路实行交通管制,2020-09-14 17:21:49
6,56天！G5京昆高速公路实行交通管制,2020-09-14 22:14:25
7,G5京昆高速西昌收费站施工 货车禁入！,2020-09-14 23:02:58
8,G5京昆高速西昌收费站施工 货车禁入！,2020-09-14 23:02:58
9,G5京昆高速公路实行交通管制 为期56天,2020-09-14 20:55:56


In [ ]:
#json_doc = json_generator(Data) 已完成写入

In [ ]:
#es.indices.delete(index='新闻', ignore=[400, 404]) 不要轻易删除

In [ ]:
#insert_batch_data(json_doc) 写过一次后不用再写